In [216]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import datetime
import re
import time
import os
import json
from PIL import Image 
import PIL 
pd.set_option('display.max_columns', None)
import random
import codecs
import subprocess

# Headers

In [2]:
headers = {
    'authority': 'www.paruvendu.fr',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"',
    'sec-ch-ua-mobile': '?0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    'cookie': 'idSessionDonneesOff=1; euconsent-v2=CPGxGkIPGxGkWDeACBFRBbCv_____3___wqIH3wAYAAgfeB98AGAAIH3gAA.f___7___4AA; _ga_K8JXZBSX4H=GS1.1.1621966262.1.1.1621966917.0; _ga=GA1.2.2578869.1621966262; __troRUID=f3900e91-e5af-40dc-8b1f-ddb816f04218; __utma=203382672.2578869.1621966262.1621966917.1621966917.1; __utmc=203382672; __utmz=203382672.1621966917.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); allreadyseen=10b2f484c9975e44eb07331aedcc1e54; pvtous=1; trackdepose=ext-google; _gid=GA1.2.931861080.1622320357; __troSYNC=1; homeallgeo=%7B%22codeInsee%22%3A%22%22%2C%22codeDepartement%22%3A%22%22%2C%22commune%22%3A%22%22%2C%22codePostal%22%3A%22%22%2C%22latitude%22%3A%22%22%2C%22longitude%22%3A%22%22%2C%22departement%22%3A%22%22%2C%22codeRegion%22%3A%22%22%2C%22codeRegionPV%22%3A%22%22%2C%22region%22%3A%22%22%2C%22distance%22%3A%22%22%2C%22idAgence%22%3A%22%22%7D; __gads=ID=d673b3e89fab95b9:T=1622387913:S=ALNI_MaCQ0JQC6C8HP-wWQjgAxU8RD-s0A; nbpvues=13; __trossion=1621966917_1800_3_f3900e91-e5af-40dc-8b1f-ddb816f04218%3A1622320357_f3900e91-e5af-40dc-8b1f-ddb816f04218%3A1622387897_1622388085_5_; datadome=FTyKvyAneBvzxZ_xbEwxmjJENZBt.UiGE5FWXfLNQKBygq4jpKiIyniEqIytT_CIGd9WFmHntGs3g8RRw.7qnavCnBpdsATzg3H_4zEeYw',
}

# Save pages x15

In [8]:
def save_page_list(req, page=1):
    datetime_1 = datetime.datetime.now().strftime("%Y-%m-%d_%Hh%M")
    page_list_name = "paruvendu" + "-" + datetime_1 + "-" + str(page)
    with open("annonces/" + page_list_name + ".html", "w")  as file:
        file.write(req.text)
        file.close()

In [95]:
pages = np.arange(60, 82)
pages

array([60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76,
       77, 78, 79, 80, 81])

In [96]:
for page_ in pages:
    req_x15 = requests.get(f"https://www.paruvendu.fr/auto-moto/listefo/default/default?moto-typeRech=&r=VMOMO000&px1=ex:%2050000&r2=&codeINSEE=&lo=&pa=&ray=100&cy=&nrj=&km1=&a0=&fulltext=&p={page_}",
                           headers = headers)
    save_page_list(req_x15, page=page_)
    time.sleep(random.randint(4, 5))
    print("x15 saved: " + str(page_))

x15 saved: 60
x15 saved: 61
x15 saved: 62
x15 saved: 63
x15 saved: 64
x15 saved: 65
x15 saved: 66
x15 saved: 67
x15 saved: 68
x15 saved: 69
x15 saved: 70
x15 saved: 71
x15 saved: 72
x15 saved: 73
x15 saved: 74
x15 saved: 75
x15 saved: 76
x15 saved: 77
x15 saved: 78
x15 saved: 79
x15 saved: 80
x15 saved: 81


# Extract data from saved x15

## Test file

In [130]:
file_check_name = "paruvendu-2021-05-30_18h19-39"
#
file_check_x15_good = f"annonces/{file_check_name}.html"

## Get prices from saved x15

In [131]:
def get_prices_from_saved_x15(r):
    
    # set empty price list
    price_list = []
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract price
    step1 = soup.select('div[class*="lazyload_bloc"]')
    if len(step1)>0:
        for k in step1:
            step2 = k.find_all("div", class_="ergov3-priceannonce")
            if len(step2)>0:
                step3 = re.findall('[0-9]', step2[0].text)
                if len(step3)>0:
                    price = float("".join(step3))
                    price_list.append(price)
                else:
                    price_list.append(np.nan)
            else:
                return None
    else:
        return None
                    
    return price_list

In [132]:
with open(file_check_x15_good, 'r') as f:
    readable_html = f.read()
    prices = get_prices_from_saved_x15(readable_html)
prices

[6650.0,
 3300.0,
 1250.0,
 1300.0,
 1000.0,
 2000.0,
 8900.0,
 800.0,
 2800.0,
 4200.0,
 3000.0,
 800.0,
 1499.0,
 12000.0,
 2200.0]

## Get references from saved x15

for this website there is no Reference on page x15 but we can still find a unique ID that will differ from announce reference

In [133]:
def get_unique_ID_from_saved_x15(r):
    
    # set empty uniq_ID list
    uniq_ID_list = []
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract uniq ID
    step1 = soup.select('div[class*="lazyload_bloc"]')
    if len(step1)>0:
        for k in step1:
            if "data-id" in k.attrs:
                uniq_ID_list.append(k.attrs["data-id"])
            else:
                uniq_ID_list.append(np.nan)
    else:
        return None
    
    return uniq_ID_list

In [134]:
with open(file_check_x15_good, 'r') as f:
    readable_html = f.read()
    unique_id = get_unique_ID_from_saved_x15(readable_html)
unique_id

['1253457316',
 '1253044073',
 '1244995184',
 '1253450850',
 '1253450767',
 '1253450229',
 '1253437517',
 '1253436054',
 '1253450065',
 '1249798628',
 '1253339878',
 '1253436525',
 '1253428712',
 '1253430608',
 '1253429389']

## Get urls from saved x15

In [135]:
def get_urls_from_saved_x15(r):
    
    # set empty url list
    url_list = []
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract url
    step1 = soup.select('div[class*="lazyload_bloc"]')
    if len(step1)>0:
        for k in step1:
            step2 = k.find_all("a")
            if len(step2)>2:
                if "href" in step2[1].attrs:
                    url_list.append(step2[1].attrs["href"])
                else:
                    url_list.append(np.nan)
            else:
                return None
    else:
        return None
    
    return url_list

In [136]:
with open(file_check_x15_good, 'r') as f:
    readable_html = f.read()
    urls_list = get_urls_from_saved_x15(readable_html)
urls_list

['https://www.paruvendu.fr/a/moto-scooter/moto/kawasaki/650-cm3/1253457316A1KVMOMOKA',
 'https://www.paruvendu.fr/a/moto-scooter/moto/yamaha/750-cm3/1253044073A1KVMOMOYA',
 'https://www.paruvendu.fr/a/moto-scooter/moto/suzuki/500-cm3/1244995184A1KVMOMOSU',
 'https://www.paruvendu.fr/a/moto-scooter/moto/yamaha/125-cm3/1253450850A1KVMOMOYA',
 'https://www.paruvendu.fr/a/moto-scooter/moto/kawasaki/125-cm3/1253450767A1KVMOMOKA',
 'https://www.paruvendu.fr/a/moto-scooter/moto/daelim/125-cm3/1253450229A1KVMOMODA',
 'https://www.paruvendu.fr/a/moto-scooter/moto/bmw/883-cm3/1253437517A1KVMOMOBM',
 'https://www.paruvendu.fr/a/moto-scooter/moto/ycf/125-cm3/1253436054A1KVMOMOYC',
 'https://www.paruvendu.fr/a/moto-scooter/moto/mz/50-cm3/1253450065A1KVMOMOMZ',
 'https://www.paruvendu.fr/a/moto-scooter/moto/bmw/1170-cm3/1249798628A1KVMOMOBM',
 'https://www.paruvendu.fr/a/moto-scooter/moto/bmw/1100-cm3/1253339878A1KVMOMOBM',
 'https://www.paruvendu.fr/a/moto-scooter/moto/ycf/125-cm3/1253436525A1KVMOM

# Download single announce pages

In [124]:
def save_page_uniq(req, uniq_id):
    datetime_1 = datetime.datetime.now().strftime("%Y-%m-%d_%Hh%M")
    page_name = "paruvendu" + "-" + uniq_id + "-" + datetime_1
    with open("pages/" + page_name + ".html", "w")  as file:
        file.write(req.text)
        file.close()

In [ ]:
for html_file in os.listdir("annonces")[60:80]:
    with open(f"annonces/{html_file}", 'r') as f:
        readable_html = f.read()
        urls_list = get_urls_from_saved_x15(readable_html)
        unique_id = get_unique_ID_from_saved_x15(readable_html)
    if urls_list != None:
        for url_single, uniq_id in zip(urls_list, unique_id):
            req_uniq = requests.get(url_single, headers = headers)
            save_page_uniq(req_uniq, uniq_id)
            print("saved page id: " + uniq_id)
            time.sleep(1)

# Process single announces
Saving scaled images x3 and adding announce to DataFrame

## DataFrame template

In [11]:
announce_template = pd.DataFrame({"url": [np.nan],
                                  "reference": [np.nan],
                                  "unique id": [np.nan],
                                  "date_scrapped": [np.nan],
                                  "announce_publication_date": [np.nan],
                                  "vehicle brand": [np.nan],
                                  "vehicle type": [np.nan],
                                  "moto scoot": [np.nan],
                                  "color": [np.nan],
                                  "vehicle condition": [np.nan],
                                  "price": [np.nan],
                                  "city": [np.nan],
                                  "postal code": [np.nan],
                                  "vehicle release date": [np.nan],
                                  "mileage": [np.nan],
                                  "Fiscal power [HP]": [np.nan],
                                  "engine capacity [CC]": [np.nan],
                                  "comments": [np.nan],
                                  "seller": [np.nan],
                                  "seller_name": [np.nan]})
announce_template

,url,reference,unique id,date_scrapped,announce_publication_date,vehicle brand,vehicle type,moto scoot,color,vehicle condition,price,city,postal code,vehicle release date,mileage,Fiscal power [HP],engine capacity [CC],comments,seller,seller_name
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## test announce

In [126]:
file_check_name_single = "paruvendu-1248033282-2021-05-30_19h04"
#
file_check_single = f"pages/{file_check_name_single}.html"

## Functions

### Get url

In [129]:
def get_url(r):
    
    # set empty url list
    url_ = ""
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract url
    step1 = soup.select('meta[property*="og:url"]')
    if len(step1)>0:
        return step1[0].attrs["content"].split("?")[0]
    else:
        return None

In [130]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_url = get_url(readable_html)
g_url

'https://www.paruvendu.fr/a/moto-scooter/moto/harley-davidson/1340-cm3/1248033282A1KVMOMOHD'

### Get reference

In [91]:
def get_reference(r):
    
    # set empty reference list
    ref_ = ""
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract ref
    step1 = soup.select('div[class*="vvdetails14_refdate"]')
    if len(step1)>0:
        return step1[0].text.replace("\n", "").replace("\t", "").split("ParuVendu")[-1].split("-")[0].strip()
    else:
        return None

In [92]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_ref = get_reference(readable_html)
g_ref

'WV167037505'

### Get unique ID

In [100]:
def get_uniq_id(url_):
    return url_.split("/")[-1].split("A1")[0]

In [101]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_url = get_url(readable_html)
get_uniq_id(g_url)

'1252519422'

### get publication date

In [211]:
def get_publication_date(r):
    
    # set empty 
    pub_ = ""
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="vvdetails14_refdate"]')
    if len(step1)>0:
        try:
            return step1[0].text.replace("\n", "").replace("\t", "").split("ParuVendu")[-1].split("-")[1].strip().split(" ")[1]
        except:
            return np.nan
    else:
        return None

In [212]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_pub = get_publication_date(readable_html)
g_pub

'20/09/2020'

### Get vehicule brand

In [34]:
def get_brand(r):
    
    # set empty 
    brand_ = ""
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[id*="blcheader"]')
    if len(step1)>0:
        return step1[0].select("h1")[0].text.replace(u'\xa0', u' ').strip().split(" ")[-1]
    else:
        return None

In [35]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_brand = get_brand(readable_html)
g_brand

'DUCATI'

### Get vehicule type

In [36]:
def get_type(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li[class*="nologo"]')
        if len(step2)>0:
            step3 = step2[0].select("span")
            if len(step3)>0:
                return step3[0].text.replace("\n", "")
    return None

In [37]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_type = get_type(readable_html)
g_type

### get moto scoot

In [38]:
def get_moto(r):
        
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[id*="blcheader"]')
    if len(step1)>0:
        return step1[0].select("h1")[0].text.replace(u'\xa0', u' ').strip().split(" ")[0]
    else:
        return None

In [39]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_moto = get_moto(readable_html)
g_moto

'MOTO'

### get color

In [40]:
def get_color(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li[class*="puiss"]')
        if len(step2)>0:
            for k in step2:
                if k.text.find("Couleur")>0:
                    return k.select("span")[0].text.strip()
    return None

In [41]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_color = get_color(readable_html)
g_color

### get vehicle condition

In [42]:
def get_cond(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li[class*="puiss"]')
        if len(step2)>0:
            for k in step2:
                if k.text.find("Etat")>0:
                    return k.select("span")[0].text.strip()
    return None

In [43]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_cond = get_cond(readable_html)
g_cond

### get fiscal power

In [44]:
def get_power(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li[class*="puiss"]')
        if len(step2)>0:
            for k in step2:
                if k.text.find("fiscale")>0:
                    return int(k.select("span")[0].text.strip())
    return None

In [45]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_power = get_power(readable_html)
g_power

### get price

In [207]:
def get_price(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[id*="autoprix"]')
    if len(step1)>0:
        try:
            return float("".join(re.findall("[0-9]", step1[0].text)))
        except:
            return np.nan
    return None

In [208]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_price = get_price(readable_html)
g_price

12500.0

### get city

In [48]:
def get_city(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[id*="blcheader"]')
    if len(step1)>0:
        return step1[0].select("h2")[0].text.split(" ")[-1].replace("\n", "")
    return None

In [49]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_city = get_city(readable_html)
g_city

'Pau'

### get postal code

In [50]:
def get_postalcode(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[id*="blcheader"]')
    if len(step1)>0:
        return step1[0].select("h2")[0].text.split(" ")[0].replace("\n", "")
    return None

In [51]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_post = get_postalcode(readable_html)
g_post

'64000'

### get vehicle release date

In [52]:
def get_releasedate(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li[class*="ann"]')
        if len(step2)>0:
            step3 = step2[0].select("span")
            if len(step3)>0:
                return "".join(re.findall("[0-9]", step3[0].text))
    return None

In [53]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_release = get_releasedate(readable_html)
g_release

'2006'

### get vehicle mileage

In [54]:
def get_mileage(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li[class*="kil"]')
        if len(step2)>0:
            step3 = step2[0].select("span")
            if len(step3)>0:
                return "".join(re.findall("[0-9]", step3[0].text))
    return None

In [55]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_mileage = get_mileage(readable_html)
g_mileage

'20500'

### get engine capacity

In [56]:
def get_capa(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li[class*="cyl"]')
        if len(step2)>0:
            step3 = step2[0].select("span")
            if len(step3)>0:
                return "".join(re.findall("[0-9]", step3[0].text))
    return None

In [57]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_capa = get_capa(readable_html)
g_capa

'992'

### get comment

In [198]:
def get_comment(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('div[class*="txt_annonceauto"]')
        if len(step2)>0:
            raw_text = step2[0].text.split("Prix")[0].strip()
            #raw_text = unicode(raw_text, errors='replace')
            return raw_text.replace(u'\x80', u' ').strip().replace("\n", " ").replace("\t", " ")
    return None

In [199]:
with codecs.open(file_check_single, 'r', encoding='utf-8', errors='ignore') as f:
    readable_html = f.read()
    g_comment = get_comment(readable_html)
g_comment

'Vends Fat Boy noir 1991 40 000 miles (64 000 km - compteur en miles).Moteur Evolution 1340cc à carburateur et allumage traditionnel (son moteur superbe : le mythique " Potatoe " Harley).Bon état général (repeint en 2011) et pneus en excellent état.Nombreux équipement additionnels :- Guidon Beach Bar de 100 et rétroviseurs after market. - Crash Bars HD.- Pots à échappement libre (mais pots d\'origine fournis en plus).- Carburateur Mikuni et filtre Hyper Charger.- Disque de frein avant récent plus disque arrière et maitres cylindres changés en juillet 2020.- Radiateur refroidissement d\'huile additionnel plus manomètre de pression et température huile.- Durites inox aviation.- Pouf passager d\'origine en cuir plus pouf amovible à ventouse fournis avec la moto.- Sacoche latérale triangulaire en cuir.- Chargeur de batterie connecté.- 2 antivols mécaniques.Possibilité d\'obtenir le certificat de véhicule de collection en août 2021 (pas de contrôle technique s\'il est mis en place pour les 

### get seller

In [60]:
def get_seller(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('p[class*="txtpresentation-vendeur"]')
    if len(step1)>0:
        if step1[0].text.find("particulier")>0:
            return ["Particulier", step1[0].text.split(":")[-1].strip().split("\n")[0].strip()]
        else:
            return ["Professionnel", step1[0].text.strip().split("\n")[0]]
    return None

In [61]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_seller = get_seller(readable_html)
g_seller

['Professionnel', 'BONNIE AND CAR']

### Save image functions

In [182]:
def get_images(r, uniq_id):
    
    # get images url list
    image_list = []
    
    # get soup
    soup_ = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup_.find('div',id="listePhotos")
    
    if step1 != None:
        if len(step1)>0:
            step2 = step1.select("img")
            if len(step2)>0:
                if "src" in step2[0].attrs:
                    for k in range(len(step2)-1):
                        image_list.append(step2[0].attrs["src"].replace("_1.jpeg", "_" + str(k+1) + ".jpeg"))

    # save images
    k=1
    for image_url in image_list[0:3]:
        image_name = f'images/{uniq_id}-{k}.jpg'
        if os.path.isfile(image_name) is False:
            img_data = requests.get(image_url).content
            with open(image_name, 'wb') as handler:
                handler.write(img_data)
            try:
                image = Image.open(image_name) 
                ratio = image.size[0] / image.size[1]
                image = image.resize((300,int(300/ratio)))
                image.save(f'images/{uniq_id}-{k}.jpg',optimize = True, quality = 50)
            except:
                pass
        k+=1
        
    return

In [183]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_ref = get_reference(readable_html)
    get_images(readable_html, g_ref)

<div style="color:orangered; font-weight:700; font-size:20pt">TEST 100 ANNONCES</div>

In [ ]:
list_test = []
for single_ in os.listdir("pages")[0:100]:
    with open(f"pages/{single_}", 'r') as f:
        readable_html = f.read()
        g_ref = get_reference(readable_html)
        get_images(readable_html, g_ref)
        print(single_)

In [576]:
#list_test

## Extract 1 announce

In [209]:
def process_announce(file_name):
    """ this function add an announce content from paruvendu.fr to the dataframe """
    
    # load csv if exists or starting from template
    data_paruvendu = "../../../tresboncoin/data/paruvendu.csv"
    #
    if os.path.isfile(data_paruvendu) is False:
        df = announce_template.copy()
    else:
        data = pd.read_csv(data_paruvendu)
        df = announce_template.copy()
    
    # fill df with defined functions
    with open(f"pages/{file_name}", 'r') as f:
        readable_html = f.read()
        
        df["url"] = get_url(readable_html)
        df["reference"] = get_reference(readable_html)
        df["unique id"] = get_uniq_id(get_url(readable_html))
        df["date_scrapped"] = datetime.datetime.now().strftime("%Y/%m/%d - %Hh%M")
        df["announce_publication_date"] = get_publication_date(readable_html)
        df["vehicle brand"] = get_brand(readable_html)
        df["vehicle type"] = get_type(readable_html)
        df["moto scoot"] = get_moto(readable_html)
        df["color"] = get_color(readable_html)
        df["vehicle condition"] = get_cond(readable_html)
        df["price"] = get_price(readable_html)
        df["city"] = get_city(readable_html)
        df["postal code"] = get_postalcode(readable_html)
        df["vehicle release date"] = get_releasedate(readable_html)
        df["mileage"] = get_mileage(readable_html)
        df["Fiscal power [HP]"] = get_power(readable_html)
        df["engine capacity [CC]"] = get_capa(readable_html)
        df["comments"] = get_comment(readable_html)
        df["seller"] = get_seller(readable_html)[0]
        df["seller_name"] = get_seller(readable_html)[1]
    
        # save images
        get_images(readable_html, get_reference(readable_html))
        
    # concatenate to csv and write
    try:
        data = pd.concat([data, df], axis=0)
        data.to_csv(path_or_buf = data_paruvendu, index=False)
    except:
        df.to_csv(path_or_buf = data_paruvendu, index=False)
    
    # deplacer le fichier html traité dans le vault
    subprocess.run(["mv", f"pages/{file_name}", "vault"])
    return

In [215]:
for single_ in os.listdir("pages"):
    process_announce(single_)

##### test

In [217]:
data_paruvendu = "../../../tresboncoin/data/paruvendu.csv"
data = pd.read_csv(data_paruvendu)
data.head(50)

,url,reference,unique id,date_scrapped,announce_publication_date,vehicle brand,vehicle type,moto scoot,color,vehicle condition,price,city,postal code,vehicle release date,mileage,Fiscal power [HP],engine capacity [CC],comments,seller,seller_name
0,https://www.paruvendu.fr/a/moto-scooter/moto/a...,WV167037505,1252519422,2021/05/31 - 23h46,13/03/2021,APRILIA,Sportive,MOTO,NaN,NaN,2500.0,Grenoble,38100,2006.0,15000.0,34.0,125.0,"APRILIA, 125 Cm3 , Sportive, Essence 2 temps, ...",Particulier,Morgan P
1,https://www.paruvendu.fr/a/moto-scooter/moto/h...,WV167315192,1253461064,2021/05/31 - 23h47,14/04/2021,HONDA,Roadster,MOTO,NaN,Bon,4970.0,Paris,75013,2014.0,13500.0,87.0,650.0,"HONDA, 650 Cm3 , Roadster, Essence 4 temps, 20...",Particulier,PIERRE D
2,https://www.paruvendu.fr/a/moto-scooter/moto/k...,WV166216457,1249557793,2021/05/31 - 23h47,01/12/2020,KTM,Roadster,MOTO,NaN,NaN,7999.0,Saint-Étienne,42000,2019.0,24600.0,125.0,1290.0,"KTM, 1290 Cm3 , Roadster, Essence 2 temps, 201...",Particulier,Jona P
3,https://www.paruvendu.fr/a/moto-scooter/moto/h...,WV166963196,1252263598,2021/05/31 - 23h47,04/03/2021,HARLEY-DAVIDSON,Custom,MOTO,NaN,NaN,17500.0,Blagnac,31700,2016.0,40000.0,16.0,1800.0,"HARLEY-DAVIDSON, 1800 Cm3 , Custom, Essence 2 ...",Particulier,Willy B
4,https://www.paruvendu.fr/a/moto-scooter/moto/b...,9926237044,1252694298,2021/05/31 - 23h47,17/05/2021,BMW,Sportive,MOTO,NaN,NaN,22500.0,Maurepas,78310,2021.0,700.0,207.0,1000.0,- Hockenheim silver metallic- Pack Dynamic : ...,Professionnel,S AUTO
5,https://www.paruvendu.fr/a/moto-scooter/moto/s...,WV166985725,1252348822,2021/05/31 - 23h47,26/05/2021,SUZUKI,Routière,MOTO,NaN,NaN,3200.0,Saint-Nazaire,44600,2006.0,36547.0,76.0,656.0,"SUZUKI Bandit S très bon état, 656 Cm3 , Routi...",Particulier,Patrick s
6,https://www.paruvendu.fr/a/moto-scooter/moto/h...,WV167130798,1252790822,2021/05/31 - 23h47,21/03/2021,HONDA,Trail,MOTO,Noir,Moyen,1150.0,Nice,6000,2001.0,98500.0,7.0,650.0,"HONDA, 650 Cm3 , Trail, Essence 4 temps, 2001,...",Particulier,Mathieu h
7,https://www.paruvendu.fr/a/moto-scooter/moto/m...,WV167033676,1252511227,2021/05/31 - 23h47,10/03/2021,MASH,Custom,MOTO,NaN,NaN,3200.0,Nimes,30900,2018.0,3000.0,5.0,400.0,"MASH, 400 Cm3 , Custom, Essence 4 temps, 2018,...",Particulier,KILIAN G
8,https://www.paruvendu.fr/a/moto-scooter/moto/p...,36673,1253056893,2021/05/31 - 23h47,18/05/2021,PIAGGIO,NaN,MOTO,NaN,NaN,7000.0,Appilly,60400,2017.0,3042.0,NaN,493.0,"Pour toute information, contactez un de nos co...",Professionnel,BONNIE AND CAR
9,https://www.paruvendu.fr/a/moto-scooter/moto/y...,WV165949859,1248699627,2021/05/31 - 23h47,10/04/2021,YAMAHA,Sportive,MOTO,NaN,NaN,2200.0,Dampjoux,25190,1996.0,19995.0,48.0,660.0,"YAMAHA, 660 Cm3 , Sportive, Essence 4 temps, 1...",Particulier,latonne


In [222]:
100 * data.isnull().sum().sort_values(ascending=False)/len(data)

color                        88.576300
vehicle condition            88.491049
Fiscal power [HP]            20.716113
vehicle type                 16.965047
mileage                       2.387042
engine capacity [CC]          2.046036
vehicle release date          1.278772
price                         0.596760
comments                      0.170503
announce_publication_date     0.170503
city                          0.085251
postal code                   0.085251
seller                        0.000000
url                           0.000000
reference                     0.000000
moto scoot                    0.000000
vehicle brand                 0.000000
date_scrapped                 0.000000
unique id                     0.000000
seller_name                   0.000000
dtype: float64